# Jupyter Notebook that will be used to support final Capstone Project for the IBM Data Certification

In [31]:
import numpy as np
import pandas as pd 
#Importing required libraries for Web Scraping
import urllib.request
from bs4 import BeautifulSoup
#import libraries for maps
import folium # map rendering library
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#libraries to manage JSON and web requests
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#import libraries for geolocation
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [32]:
print('Hello Capstone Project Course!!')

Hello Capstone Project Course!!


# Week-3 Assignement

## Part 1 Scraping the Web - Wikipidia Page

In [33]:
#defining the URL for the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [34]:
#Using Beatiful Soup library to get the html information
Toronto_page = urllib.request.urlopen(url)
soup = BeautifulSoup(Toronto_page, "html.parser")
print(soup.prettify())
#Saving html object to txt file
with open('url.txt', 'w', encoding='utf-8') as f_out:
    f_out.write(soup.prettify())

 "
      <a dir="ltr" href="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&amp;oldid=995657573">
       https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&amp;oldid=995657573
      </a>
      "
     </div>
    </div>
    <div class="catlinks" data-mw="interface" id="catlinks">
     <div class="mw-normal-catlinks" id="mw-normal-catlinks">
      <a href="/wiki/Help:Category" title="Help:Category">
       Categories
      </a>
      :
      <ul>
       <li>
        <a href="/wiki/Category:Communications_in_Ontario" title="Category:Communications in Ontario">
         Communications in Ontario
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Postal_codes_in_Canada" title="Category:Postal codes in Canada">
         Postal codes in Canada
        </a>
       </li>
       <li>
        <a href="/wiki/Category:Toronto" title="Category:Toronto">
         Toronto
        </a>
       </li>
       <li>
        <a href="/wi

In [35]:
#extracting the table from the BeatifulSoup object
city_table=soup.find("table",class_='wikitable sortable') #this is the class and tag for the zip code table
#extracting the headers from the table
table_header =city_table.find_all('th')


In [36]:
#Creating Arrays to store the data and roe headers from the table

A=[]
B=[]
C=[]
Header = []
# finding header in the table <th> tag
header_table = city_table.find_all('th')
# Append the header string values to the Header Array
for i,item in enumerate(header_table):
    
    Header.append(header_table[i].string.strip('\n'))
   
# Append the row values to the Arrays

for row in city_table.find_all('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True).strip('\n'))
        B.append(cells[1].find(text=True).strip('\n'))
        C.append(cells[2].find(text=True).strip('\n'))
    

        

In [37]:
#creating DataFrame from extracted values
df = pd.DataFrame({Header[0]:A,Header[1]:B,Header[2]:C})
df=df[df["Borough"]!="Not assigned"]
df.reset_index(inplace=True,drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Part Two: Getting latidu & Longitude data

In [38]:
#using the csv file, since the geolocator option did not work for me

urlLoc = 'http://cocl.us/Geospatial_data'
dfLoc = pd.read_csv(urlLoc)
dfLoc.set_index('Postal Code',inplace = True)
dfLoc.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [39]:
#mergin both dataframes based on the postal code index
df2 = df.set_index('Postal Code')
df2 = df2.merge(dfLoc,how='left',left_index=True,right_index=True)
df2.sort_index(inplace=True)
df2.reset_index(inplace = True)
df2.head(20)



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part 3: clustering




In [40]:
# creating Dataframes that contain only Buoroughs that contain the word Toronto
Toronto_Buroughs_df = df2[df2['Borough'].str.contains('Toronto')]
Toronto_Buroughs_df.rename(columns={'Neighbourhood':'Neighborhood'},inplace = True)
Toronto_Buroughs_df.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [41]:
#  Use geopy library to get the latitude and longitude values of New York City.
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [42]:
# create map of Toronto using latitude and longitude values
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for latTR, lngTR, labelTR in zip(Toronto_Buroughs_df['Latitude'], Toronto_Buroughs_df['Longitude'], Toronto_Buroughs_df['Neighborhood']):
    label = folium.Popup(labelTR, parse_html=True)
    folium.CircleMarker(
        [latTR, lngTR],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map 

## Update your FourSquare credentials

In [43]:

# Update your credentials to 
CLIENT_ID = 'K3NJQJUFFXECQSBF5OV02FKWGBBK5TMSMNP44VA01AUZ2GPY' # your Foursquare ID
CLIENT_SECRET = '0UW2QVVVS21JMLI0JOQJOPFO30WRUTCXT3ZL4WYIWGAUXYAT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K3NJQJUFFXECQSBF5OV02FKWGBBK5TMSMNP44VA01AUZ2GPY
CLIENT_SECRET:0UW2QVVVS21JMLI0JOQJOPFO30WRUTCXT3ZL4WYIWGAUXYAT


In [44]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [45]:
# function to repeat the same process to all the neighborhoods in Toronto dataset
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
# get all venues in the Toronto dataset
Toronto_venues = getNearbyVenues(names=Toronto_Buroughs_df['Neighborhood'],
                                   latitudes=Toronto_Buroughs_df['Latitude'],
                                   longitudes=Toronto_Buroughs_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [47]:
#check the resultin dataframe
print(Toronto_venues.shape)
Toronto_venues.tail()

(1618, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1613,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1614,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1615,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1616,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
1617,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center


In [48]:
# check the number of venues per neighborhooh
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [49]:
#count uniqu categories for each neighborhood

print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [50]:
#Analyze Each Neighborhood
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.062500,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0000,0.0000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015625,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625
5,Christie,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025000,0.0125,0.0125,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.012500,0.0125,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.012500,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0000,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [52]:
#Function to define the most common venues in df

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Bakery,Restaurant,Seafood Restaurant,Butcher
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Bar,Italian Restaurant,Restaurant,Climbing Gym,Burrito Place
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Sculpture Garden,Plane,Rental Car Location,Boat or Ferry,Bar,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Salad Place,Bubble Tea Shop,Thai Restaurant,Ramen Restaurant,Portuguese Restaurant


In [54]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
      dtype=int32)

In [55]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_Buroughs_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Trail,Health Food Store,Asian Restaurant,Wine Bar,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Grocery Store,Pub,Pizza Place,Lounge
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Sandwich Place,Park,Fast Food Restaurant,Liquor Store,Fish & Chips Shop,Pub,Ice Cream Shop,Sushi Restaurant,Burrito Place,Italian Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,American Restaurant,Brewery,Gastropub,Café,Bakery,Yoga Studio,Ice Cream Shop,Bank,Bar
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Construction & Landscaping,Bus Line,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [58]:
# Let's examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories.

Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Pub,Trail,Health Food Store,Asian Restaurant,Wine Bar,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Grocery Store,Pub,Pizza Place,Lounge
42,East Toronto,0,Sandwich Place,Park,Fast Food Restaurant,Liquor Store,Fish & Chips Shop,Pub,Ice Cream Shop,Sushi Restaurant,Burrito Place,Italian Restaurant
43,East Toronto,0,Coffee Shop,American Restaurant,Brewery,Gastropub,Café,Bakery,Yoga Studio,Ice Cream Shop,Bank,Bar
45,Central Toronto,0,Hotel,Park,Pizza Place,Food & Drink Shop,Sandwich Place,Department Store,Breakfast Spot,Gym / Fitness Center,Doner Restaurant,Deli / Bodega
46,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Pet Store,Chinese Restaurant,Café
47,Central Toronto,0,Sandwich Place,Dessert Shop,Sushi Restaurant,Italian Restaurant,Coffee Shop,Gym,Thai Restaurant,Café,Pizza Place,Brewery
49,Central Toronto,0,Coffee Shop,Pizza Place,Liquor Store,Restaurant,Bank,Pub,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint
51,Downtown Toronto,0,Pizza Place,Coffee Shop,Restaurant,Pub,Italian Restaurant,Bakery,Café,Grocery Store,Chinese Restaurant,Sandwich Place
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Café,Men's Store,Mediterranean Restaurant,Hotel


In [117]:
#Explore unique values in each category to define clustering name

Toronto_unique = Toronto_merged.drop(['Borough','Postal Code','Neighborhood','Latitude','Longitude'],axis=1).drop_duplicates()

for i,item in enumerate(Toronto_unique['Cluster Labels'].unique()):
    for j,item2 in enumerate(Toronto_unique[Toronto_unique['Cluster Labels']==i].columns):
        print(Toronto_unique[Toronto_unique['Cluster Labels']==i].iloc[:,j].unique())
    
    

[0]
['Pub' 'Greek Restaurant' 'Sandwich Place' 'Coffee Shop' 'Hotel'
 'Clothing Store' 'Pizza Place' 'Jewelry Store' 'Café' 'Airport Service'
 'Grocery Store' 'Pharmacy' 'Bar' 'Gift Shop' 'Light Rail Station']
['Trail' 'Coffee Shop' 'Park' 'American Restaurant' 'Dessert Shop'
 'Pizza Place' 'Japanese Restaurant' 'Café' 'Clothing Store'
 'Cocktail Bar' 'Italian Restaurant' 'Aquarium' 'Hotel' 'Restaurant' 'Bar'
 'Vegetarian / Vegan Restaurant' 'Airport Terminal' 'Seafood Restaurant'
 'Bakery' 'Mexican Restaurant' 'Breakfast Spot' 'Sushi Restaurant'
 'Yoga Studio']
['Health Food Store' 'Italian Restaurant' 'Fast Food Restaurant' 'Brewery'
 'Pizza Place' 'Yoga Studio' 'Sushi Restaurant' 'Liquor Store'
 'Restaurant' 'Park' 'Café' 'American Restaurant' 'Beer Bar'
 'Sandwich Place' 'Hotel' 'Mexican Restaurant' 'Coffee Shop'
 'Vietnamese Restaurant' 'Airport Lounge' 'Grocery Store'
 'Vegetarian / Vegan Restaurant' 'Breakfast Spot' 'Thai Restaurant'
 'Auto Workshop']
['Asian Restaurant' 'Furnit